In [9]:
from pyspark.context import SparkContext
from pyspark import SparkConf,sql
from pyspark.sql.session import SparkSession
import findspark
import pymysql
import pymysql.cursors
import pandas as pd
from pyspark.sql.functions import *

In [2]:
product_path='D:/PROJECTS/Recomendation_System_FP/Code/Database/product.csv'
cosin_result_path2='D:/PROJECTS/Recomendation_System_FP/cosin_simmilarity2.csv'#'D:/PROJECTS/Recomendation_System_FP/cosin_simmilarity1.csv'
cosin_result_path1='D:/PROJECTS/Recomendation_System_FP/cosin_simmilarity1.csv'

In [3]:
spark = SparkSession.builder \
    .appName("PySpark MySQL Example") \
    .config("spark.driver.extraClassPath", "C:/spark/spark-3.5.0-bin-hadoop3/jars/mysql-connector-java-5.1.46.jar") \
    .getOrCreate()



In [71]:
Product_df=spark.read.format('csv').option('header',True) \
.load(product_path)

cosin_result_path_df1=spark.read.format('csv').option('header',True) \
.load(cosin_result_path1)

cosin_result_path_df2=spark.read.format('csv').option('header',True) \
.load(cosin_result_path2)

In [72]:
cosin_result_path_df1=cosin_result_path_df1.select('*',round('cosin_score',3).alias('cosin_score_1')).withColumnRenamed('Product_A','original_product').withColumnRenamed('Product_B','matched_product')

In [73]:
cosin_result_path_df1=cosin_result_path_df1.toPandas()

In [74]:
cosin_result_path_df1=cosin_result_path_df1.drop(columns=['cosin_score','_c0'],axis=1).rename(columns={'cosin_score_1':'cosin_score'})

In [76]:
cosin_result_path_df1.to_csv('validated_cosin_simmilarity.csv')

In [30]:
cosin_result_path_df1=cosin_result_path_df1.select(['original_product','matched_product','round(cosin_score, 3)'])

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `original_product` cannot be resolved. Did you mean one of the following? [` original_product`, `matched_product`, `cosin_score`, `cosin_score_1`, `_c0`].;
'Project ['original_product, matched_product#601, round(cosin_score, 3)#632]
+- Project [_c0#553, cosin_score#554,  original_product#595, matched_product#601, cosin_score_1#588, round(cast(cosin_score#554 as double), 3) AS round(cosin_score, 3)#632]
   +- Project [_c0#553, cosin_score#554,  original_product#595, Product_B#556 AS matched_product#601, cosin_score_1#588]
      +- Project [_c0#553, cosin_score#554, Product_A#555 AS  original_product#595, Product_B#556, cosin_score_1#588]
         +- Project [_c0#553, cosin_score#554, Product_A#555, Product_B#556, round(cast(cosin_score#554 as double), 3) AS cosin_score_1#588]
            +- Relation [_c0#553,cosin_score#554,Product_A#555,Product_B#556] csv


In [5]:
product_title='WILDWOOD RUFFLED TOP'
selected_product_id=Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]
recomendation_df_product=cosin_result_path_df1[cosin_result_path_df1['Product_A']==selected_product_id]
link=[]
for row in recomendation_df_product.collect():
    # if float(row['cosin_score'])>=.80:
        id=row['Product_B']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
             link.append(id)
        except:
            print('valid string')    
print(','.join(link))

3842,2010,262,3155,579,5157,3611,2874,1493,324,946,5004,3277


In [6]:
product_title='WILDWOOD RUFFLED TOP'
selected_product_id=Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]
recomendation_df_product=cosin_result_path_df2[cosin_result_path_df2['Product_A']==selected_product_id]
link=[]
for row in recomendation_df_product.collect():
    # if float(row['cosin_score'])>=.80:
        id=row['Product_B']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
             link.append(id)
        except:
            print('valid string')    
print(','.join(link))

3842,2548,2301,3942,2010,8595,2707,262,2146,3155,5157,7773,529,3611,2874,1493,324,226,946,5004,7507,3979,3277,699,6068,2854
